# Get Started

In [ ]:
# @title Install Libraries
!pip -q install pinecone
!pip -q install langchain_community
!pip -q install langchain_openai
!pip -q install langchain_pinecone
!pip -q install langchain-huggingface
!pip -q install langsmith

In [ ]:
# @title Import Libraries
import os
from google.colab import userdata
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEndpointEmbeddings
from langchain_openai import ChatOpenAI
from langchain.prompts import load_prompt
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from langsmith import Client, traceable
import yaml
import inspect
import json

In [ ]:
# @title Set Environment Variables
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_PROJECT'] = userdata.get('PINECONE_INDEX_NAME')
os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['OPENAI_API_BASE'] = 'https://openrouter.ai/api/v1'
os.environ['OPENAI_API_KEY'] = userdata.get('OPENROUTER_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
os.environ['PINECONE_INDEX_NAME'] = userdata.get('PINECONE_INDEX_NAME')

# Method

In [ ]:
# @title Get Pinecone Index
def get_index(index_name=os.environ['PINECONE_INDEX_NAME']):
  pc = Pinecone()
  if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=4096,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

  return pc.Index(index_name)

In [ ]:
# @title Text Splitter
def init_text_splitter(chunk_size=1000, chunk_overlap=0):
  return CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

In [ ]:
# @title Add Documents to Vector Store
def add_documents(path):
  loader = TextLoader(path)
  documents = loader.load()
  print(f"Loaded {len(documents)} document(s) from {path}")
  docs = text_splitter.split_documents(documents)
  vectorstore.add_documents(docs)
  print(f"Added {len(docs)} text chunk(s) to vector store.")

In [ ]:
# @title Add Texts to Vector Store
def add_texts(text):
  texts = text_splitter.split_text(text)
  vectorstore.add_texts(texts)
  print(f"Added {len(texts)} text chunks to vector store.")

In [ ]:
# @title Initialize Components

index = get_index()

llm = ChatOpenAI(model='qwen/qwen3-8b:free')

embeddings = HuggingFaceEndpointEmbeddings(model='Qwen/Qwen3-Embedding-8B')

vectorstore = PineconeVectorStore(index=index, embedding=embeddings)

text_splitter = init_text_splitter()

retriever = vectorstore.as_retriever(k=6)

qa_chain = RetrievalQA.from_chain_type(
  llm,
  retriever=retriever,
  return_source_documents=True
)

# Evaluators

In [ ]:
query = "công ty này ở phường nào"
response = qa_chain.invoke({"query": query})

print(response['source_documents'])

[Document(id='46b8f2d3-955d-4652-82d6-b98c9b66f8ac', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/public/RAG_ChatBot/dataset/introduction.txt'}, page_content='Mã số thuế\t\n2301025890 - Ngày cấp: 17/04/2018\nTên đơn vị\t\nCÔNG TY TNHH THƯƠNG MẠI VÀ ĐẦU TƯ TỔNG HỢP ANH PHÁT\nĐịa chỉ theo CQT\t\nNR ông Nguyễn Văn Trường, xóm Rừng, khu Bồ Sơn, Phường Võ Cường, Tỉnh Bắc Ninh, Việt Nam\nĐịa chỉ sau sáp nhập\t\nHệ thống tìm thấy 1 kết quả địa chỉ mới liên quan của MST 2301025890:\n\n- Địa chỉ 1: NR ông Nguyễn Văn Trường, xóm Rừng, khu Bồ Sơn, Phường Võ Cường, Tỉnh Bắc Ninh, Việt Nam\n\n- Căn cứ:\n\nPhường Võ Cường: Sắp xếp toàn bộ diện tích tự nhiên, quy mô dân số của các phường Đại Phúc, Phong Khê và Võ Cường thành phường mới có tên gọi là phường Võ Cường.\n(Thông tin mang tính tham khảo, để có thông tin chính xác vui lòng tra cứu từ website của Cục thuế hoặc Cổng thông tin doanh nghiệp quốc gia trước khi lập/xuất hóa đơn, chứng từ điện tử)\n\nTrạng thái\tNNT đang hoạt động

In [ ]:
root_path =  userdata.get('ROOT_PATH')
dataset_path = f'{root_path}/datasets'
evaluators_path = f'{root_path}/evaluators'

In [ ]:
dataset_name='FQA'
client = Client()

examples = json.load(open(f'{dataset_path}/evaluation.json'))

if not client.has_dataset(dataset_name=dataset_name):
  dataset = client.create_dataset(dataset_name=dataset_name)
  client.create_examples(
      dataset_id=dataset.id,
      examples=examples
  )


In [ ]:
def init_grader_llm(model='qwen/qwen3-8b:free', temperature=0, scope=None):
  return ChatOpenAI(model=model, temperature=temperature).with_structured_output(
    yaml.safe_load(open(f'{evaluators_path}/{scope}/output_schema.yaml')),
    method='json_schema', strict=True
  )

In [ ]:
# @title Correctness
correctness_path=f'{evaluators_path}/correctness'



def correctness(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
  """An evaluator for RAG answer accuracy"""
  grader_llm = ChatOpenAI(model='qwen/qwen3-8b:free', temperature=0).with_structured_output(
    yaml.safe_load(open(f'{correctness_path}/output_schema.yaml')),
    method="json_schema", strict=True
  )

  instructions = load_prompt(f'{correctness_path}/system_prompt.yaml').format()

  answers = load_prompt(f'{correctness_path}/user_prompt.yaml').format(
    question=inputs['question'],
    reference=reference_outputs['answer'],
    answer=outputs['answer']
  )

  grade = grader_llm.invoke([
    {"role": "system", "content": instructions},
    {"role": "user", "content": answers}
  ])
  return grade["correct"]


In [ ]:
# Add decorator so this function is traced in LangSmith
@traceable()
def rag_bot(question: str) -> dict:

  instructions = f"""You are a helpful assistant who is good at analyzing source information and answering questions.
      Use the following source documents to answer the user's questions.
      If you don't know the answer, just say that you don't know.
      Use three sentences maximum and keep the answer concise.

"""
  # langchain ChatModel will be automatically traced
  ai_msg = qa_chain.invoke([
          {"role": "system", "content": instructions},
          {"role": "user", "content": question},
      ],
  )
  return {"answer": ai_msg['result'], "documents": ai_msg['source_documents']}

In [ ]:
def target(inputs: dict) -> dict:
    return rag_bot(inputs["question"])

In [ ]:
experiment_results = client.evaluate(
    target,
    data='FQA',
    evaluators=[correctness],#, groundedness, relevance, retrieval_relevance],
    experiment_prefix="rag-doc-relevance"
)

View the evaluation results for experiment: 'rag-doc-relevance-46bb454b' at:
https://smith.langchain.com/o/00fa7863-e8c4-446a-9682-0dec08610f9c/datasets/ae8be8f8-25cb-40c3-aeae-5ef8b9c93069/compare?selectedSessions=1c1bb288-086d-4afe-8fa1-0d33c0c0957f




0it [00:00, ?it/s]

{'query': 'What are the types of biases that can arise with few-shot prompting?', 'result': "The information provided in the context is about a company's tax registration details in Vietnam and does not contain any content related to machine learning, biases, or few-shot prompting. Therefore, I cannot answer the question based on the given context. If you have other questions about the company or its information, I can assist with that.", 'source_documents': [Document(id='46b8f2d3-955d-4652-82d6-b98c9b66f8ac', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/public/RAG_ChatBot/dataset/introduction.txt'}, page_content='Mã số thuế\t\n2301025890 - Ngày cấp: 17/04/2018\nTên đơn vị\t\nCÔNG TY TNHH THƯƠNG MẠI VÀ ĐẦU TƯ TỔNG HỢP ANH PHÁT\nĐịa chỉ theo CQT\t\nNR ông Nguyễn Văn Trường, xóm Rừng, khu Bồ Sơn, Phường Võ Cường, Tỉnh Bắc Ninh, Việt Nam\nĐịa chỉ sau sáp nhập\t\nHệ thống tìm thấy 1 kết quả địa chỉ mới liên quan của MST 2301025890:\n\n- Địa chỉ 1: NR ông Nguyễn Văn Trường, x